bibliotecas

In [1]:
!pip install -r "requirements.txt"

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


codigo

In [ ]:
import yt_dlp
import speech_recognition as sr
import os
from pydub import AudioSegment
import math

def download_youtube_video(url, output_path='.', index=0):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/audio_{index}.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f'Download completo para URL {index + 1}.')
    except Exception as e:
        print(f'Erro ao baixar o vídeo {index + 1}: {e}')

def split_audio(audio_file, chunk_length_ms=60000, start_minute=2):
    audio = AudioSegment.from_file(audio_file)
    chunks = []

    # Calcula o ponto de início em milissegundos
    start_ms = start_minute * 60 * 1000

    # Inicia a partir de start_ms
    for i in range(start_ms, len(audio), chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        chunks.append(chunk)
    return chunks

def transcribe_audio_chunk(chunk, recognizer):
    try:
        with chunk.export(format="wav") as temp_wav:
            with sr.AudioFile(temp_wav) as source:
                audio = recognizer.record(source)
            return recognizer.recognize_google(audio, language="pt-BR")
    except sr.UnknownValueError:
        return "Desculpe, não consegui entender o áudio."
    except sr.RequestError as e:
        print(f'Erro na solicitação; verifique a conexão com a internet: {e}')
        return "Erro na solicitação; verifique a conexão com a internet."

def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    chunks = split_audio(audio_file)
    transcription = ""
    for i, chunk in enumerate(chunks):
        print(f'Transcrevendo parte {i + 1}/{len(chunks)}')
        transcription += transcribe_audio_chunk(chunk, recognizer) + " "
    print('Transcrição completa.')
    return transcription.strip()

def save_transcription_to_file(transcription, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(transcription)
        print(f'Transcrição salva em {file_path}')
    except Exception as e:
        print(f'Erro ao salvar a transcrição: {e}')

def process_video_links(url_list, output_path='.'):
    for index, url in enumerate(url_list):
        print(f'Processando URL {index + 1}/{len(url_list)}: {url}')
        audio_file = os.path.join(output_path, f'audio_{index}.wav')
        transcription_file = os.path.join(output_path, f'transcription_{index}.txt')
        
        # Passos para baixar, extrair áudio e transcrever
        download_youtube_video(url, output_path, index)

        # Verificar se o arquivo de áudio foi criado
        if os.path.exists(audio_file):
            transcription = transcribe_audio(audio_file)
            # Salvar a transcrição em um arquivo de texto
            save_transcription_to_file(transcription, transcription_file)
        else:
            print(f'O arquivo de áudio {index + 1} não foi encontrado.')

# Lista de URLs do YouTube
link = ['https://www.youtube.com/watch?v=IO28g9ZBeh4&t=3s']

# Caminho para salvar os arquivos
output_path = '.'

# Processar a lista de vídeos
process_video_links(link, output_path)